In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from functions import *

In [47]:
device = torch.device('cpu')
word_dict = load_pickle(   '../data/word_dict.pkl' )
seq_train = load_pkl2ndarr( os.path.join('../data/ot_input/', 'OT'+'_train_proteins.pkl') )
target_train = load_pkl2ndarr( os.path.join('../data/ot_input/', 'OT'+'_train_targets.pkl') )
datapack = [seq_train, target_train]
batch_data = [datapack[di][8: 12] for di in range(len(datapack))]
words, target_values = data2tensor(batch_data, True, device)

In [ ]:
# embeddding-attention-cnn1d-dense layer

In [48]:
#embedding
dim=64
embed_word = nn.Embedding(len(word_dict), dim)
x = embed_word(words)
print(x.size())

torch.Size([4, 554, 64])


In [49]:
#self attention
key= nn.Linear(dim, dim)
query= nn.Linear(dim, dim)
value = nn.Linear(dim, dim)
h1= torch.relu(key(x))
h2= torch.relu(query(x))
h3 = torch.relu(value(x))
A = F.softmax( torch.matmul(h1,h2.transpose(1,2) ), dim=-1)
y = torch.matmul(A,h3)
print(A.size())
print(y.size())

torch.Size([4, 554, 554])
torch.Size([4, 554, 64])


In [50]:
window = 5
cnn = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2*window+1, stride=1, padding=window)

In [51]:
y=torch.unsqueeze(y, 1)
y = torch.relu(cnn(y)) 
y = torch.squeeze(y, 1)

In [55]:
m = nn.AdaptiveAvgPool2d((1, 64))
k = torch.squeeze( m(y), 1)
k.size()

torch.Size([4, 64])

In [39]:
out = torch.max(y, 1).values
out.size()

torch.Size([4, 64])

In [43]:
out_layer = nn.Linear( dim , 1)

In [44]:
final_out =  out_layer (out)
final_out.size()

torch.Size([4, 1])

In [45]:
final_out

tensor([[0.0990],
        [0.1098],
        [0.0652],
        [0.0807]], grad_fn=<AddmmBackward>)